In [46]:
import pandas as pd
import numpy as np
import csv
import json

# Cleaning

In [2]:
# read in data
df_list = []
for chunk in pd.read_json('yelp_academic_dataset_business.json', chunksize=200, lines=True):
    df_list.append(chunk)
df = pd.concat(df_list)

In [3]:
# clean table for faster loading
df.dropna(subset=['categories','hours','state'], inplace=True)
df = df[df['categories'].str.contains("Restaurants")]
df = df[df['is_open']==1]
df = df[~df['state'].isin(['ON','AB','QC','MB','BC'])]
df = df[~df['postal_code'].str.contains(" ")]
df = df.drop(columns=['attributes'])
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"Ethnic Food, Food Trucks, Specialty Food, Impo...","{'Monday': '11:30-14:30', 'Tuesday': '11:30-14..."
33,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,"4550 East Cactus Rd, #KSFC-4",Phoenix,AZ,85032,33.602822,-111.983533,4.0,10,1,"Food, Pretzels, Bakeries, Fast Food, Restaurants","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
41,98hyK2QEUeI8v2y0AghfZA,Pho Lee's Vietnamese Restaurant,"1541 E 38th St, Ste 101",Cleveland,OH,44114,41.512155,-81.663332,4.5,23,1,"Restaurants, Vietnamese, Soup","{'Monday': '11:0-20:0', 'Wednesday': '11:0-20:..."
49,tLpkSwdtqqoXwU0JAGnApw,Wendy's,4602 Northfield Road,Cleveland,OH,44128,41.434614,-81.527026,3.5,7,1,"Restaurants, Fast Food, Burgers","{'Monday': '10:0-3:0', 'Tuesday': '10:0-3:0', ..."
54,lK-wuiq8b1TuU7bfbQZgsg,Hingetown,,Cleveland,OH,44113,41.489343,-81.711029,3.0,4,1,"Shopping Centers, Food, Coffee & Tea, Cafes, M...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [5]:
# # export to csv: for python script
# df.to_csv('yelp_business.csv',index=False)

# Cassandra

In [5]:
# # import from csv: for python script
# df = pd.read_csv('yelp_business.csv')
# df.head()

In [4]:
# start cassandra cluster
from cassandra.cluster import Cluster
cluster = Cluster(['cassandra'])
session = cluster.connect()

In [5]:
# make keyspace
session.execute(
    "CREATE KEYSPACE IF NOT EXISTS ks "
    "WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 };"
)

session.set_keyspace('ks')

In [6]:
# session.execute("""
# DROP TABLE business;
# """)

In [9]:
# make table
session.execute("""
CREATE TABLE business(
   id text,
   name text,
   address text,
   city text,
   state text,
   stars float,
   review_count int,
   is_open int,
   categories text,
   hours text,
   PRIMARY KEY (id, name)
);
""")

OperationTimedOut: errors={'Connection defunct by heartbeat': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=10.40.224.55:9042

In [11]:
# insert dataframe into table
query = "INSERT INTO business (id, name, address, city, state, stars, review_count, is_open, categories, hours)  VALUES (?,?,?,?,?,?,?,?,?,?)"
prepared = session.prepare(query)

for index, item in df.iterrows():
    session.execute(prepared, (item[0], item[1], str(item[2]), item[3], item[4], item[8], item[9], item[10], item[11], str(item[12])))
    

WriteTimeout: Error from server: code=1100 [Coordinator node timed out waiting for replica nodes' responses] message="Operation timed out - received only 0 responses." info={'consistency': 'LOCAL_ONE', 'required_responses': 1, 'received_responses': 0, 'write_type': 'SIMPLE'}

In [19]:
# make queries on state
query = """
SELECT id, name, address, city, state, stars, review_count, is_open, categories, hours
FROM business
WHERE state = %s and is_open = 1
ALLOW FILTERING;
"""

rows = session.execute(query, ('AZ', )) # input example AZ for state

# export query results to csv
header = ['id','name', 'address', 'city', 'state', 'stars', 'review_count', 'categories', 'hours']
cdf = pd.DataFrame(columns=header)

for row in rows:
    line = [row.id, row.name, row.address, row.city, row.state, row.stars, row.review_count, row.categories, row.hours]
    cdf = pd.concat([pd.DataFrame([line],columns=cdf.columns), cdf])

# with open('test.csv', 'w') as fp:
#     writer = csv.writer(fp, delimiter=',')
#     writer.writerow(header)
#     for row in rows:
#         writer.writerow([row.id, row.name, row.address, row.city, row.state, row.stars, row.review_count, row.categories, row.hours])

In [28]:
cdf = cdf.reset_index(drop=True)
cdf

,id,name,address,city,state,stars,review_count,categories,hours
0,LGEIsxeJQATo9J1IA-TEdA,Fajitas A Sizzlin Celebration,9841 N Black Canyon Hwy,Phoenix,AZ,3.5,328,"Salad, Soup, Restaurants, Tex-Mex, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
1,-i0DZp5-KT54MpqvREmqXQ,Pho Tan Thanh,7505 W Cactus Rd,Peoria,AZ,3.5,92,"Ethnic Food, Vietnamese, Specialty Food, Resta...","{'Monday': '9:0-21:0', 'Wednesday': '9:0-21:0'..."
2,AcjO26DmOhqWlfo2s5DUow,Stackers Restaurant,"2855 W Cactus Rd, Ste 24",Phoenix,AZ,3.5,83,"American (Traditional), Nightlife, Pubs, Bars,...","{'Monday': '11:0-1:0', 'Tuesday': '11:0-1:0', ..."
3,zJGtD3y-pAIGNId4codEEg,Otro Cafe,6035 N 7th St,Phoenix,AZ,4.0,808,"Restaurants, Mexican, Breakfast & Brunch, Cafes","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ..."
4,X54yOvzhEKyAurDOeQ-BkA,QDOBA Mexican Eats,"3300 W Camelback Rd, Thunder Alley Bldg",Phoenix,AZ,3.0,6,"Fast Food, Restaurants, Mexican, Caterers, Eve...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-23:0', '..."
...,...,...,...,...,...,...,...,...,...
718,b1m_PK-ggGR3CmBjdIyqkQ,Wrights at The Arizona Biltmore,2400 E Missouri Ave,Phoenix,AZ,4.0,139,"Restaurants, American (New), Breakfast & Brunch","{'Monday': '17:30-21:30', 'Tuesday': '17:30-21..."
719,cCX_cpuuzTI3lDUyKFCnAg,Filiberto's Mexican Food,13972 N. 83rd Ave,Peoria,AZ,3.0,99,"Mexican, Restaurants, Breakfast & Brunch, Salad","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
720,64E4jIkHN20RVJoaM2fyGw,O'Kelley's,"2120 W Guadalupe Rd, Ste 17",Mesa,AZ,3.5,72,"Arts & Entertainment, Music Venues, Nightlife,...","{'Monday': '6:0-2:0', 'Tuesday': '6:0-2:0', 'W..."
721,rMCKv9puuu2NkYISHnwyWg,Rubio's,"4747 E Bell Rd, Ste 17",Phoenix,AZ,3.5,100,"Restaurants, Seafood, Event Planning & Service...","{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:..."


In [21]:
# drop table 
session.execute("""
DROP TABLE business;
""")

# Redis

In [36]:
# start redis session
import redis
r = redis.Redis(host='my-redis', port=6379, db=0,decode_responses=True)

In [22]:
# load query result from cassandra
#cdf = pd.read_csv('test.csv')
#cdf

In [37]:
# populate redis with query result
pipe = r.pipeline()

for i in range(len(cdf)):
    id_ = i
    pipe.hmset(id_, {'business_id': cdf['id'][i],
                     'name': cdf['name'][i],
                     'address': cdf['address'][i],
                     'city': cdf['city'][i], 
                     'state': cdf['state'][i],
                     'stars': cdf['stars'][i],
                     'review_count': cdf['review_count'][i],
                     'categories': cdf['categories'][i],
                     'hours': cdf['hours'][i]})
    pipe.zadd(cdf['city'][i], {id_:i})
pipe.execute()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Pipeline.hmset() is deprecated. Use Pipeline.hset() instead.
  


[True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,
 True,
 1,

In [47]:
# query on city; print out output to a JSON file
with open('test.json', 'w') as f:
    for x in r.zrangebyscore('Phoenix', 0, len(cdf)):
        pipe.hgetall(x)
    json.dump(pipe.execute(),f)

# Check Output

In [48]:
test_list = []
test = pd.read_json('test.json')
test

,state,categories,stars,business_id,review_count,name,address,city,hours
0,AZ,"Salad, Soup, Restaurants, Tex-Mex, Mexican",3.5,LGEIsxeJQATo9J1IA-TEdA,328,Fajitas A Sizzlin Celebration,9841 N Black Canyon Hwy,Phoenix,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
1,AZ,"American (Traditional), Nightlife, Pubs, Bars,...",3.5,AcjO26DmOhqWlfo2s5DUow,83,Stackers Restaurant,"2855 W Cactus Rd, Ste 24",Phoenix,"{'Monday': '11:0-1:0', 'Tuesday': '11:0-1:0', ..."
2,AZ,"Restaurants, Mexican, Breakfast & Brunch, Cafes",4.0,zJGtD3y-pAIGNId4codEEg,808,Otro Cafe,6035 N 7th St,Phoenix,"{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ..."
3,AZ,"Fast Food, Restaurants, Mexican, Caterers, Eve...",3.0,X54yOvzhEKyAurDOeQ-BkA,6,QDOBA Mexican Eats,"3300 W Camelback Rd, Thunder Alley Bldg",Phoenix,"{'Monday': '0:0-0:0', 'Tuesday': '7:0-23:0', '..."
4,AZ,"American (Traditional), Bars, Tex-Mex, Nightli...",2.5,2SHu52w_w4Z4ACq-vx3a-Q,78,Chili's,2612 N 75th Ave,Phoenix,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
...,...,...,...,...,...,...,...,...,...
227,AZ,"Food, Mexican, Restaurants",3.0,ERM603jbIbNqX2c7Ww1Qiw,89,Cafe Rio,12005 N Tatum Blvd,Phoenix,"{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:..."
228,AZ,"Pizza, Restaurants",3.0,8F9C5vhDlYphBWELUgLkvQ,94,Famous Ray's Pizza,2501 W Happy Valley Rd,Phoenix,"{'Monday': '10:0-21:30', 'Tuesday': '10:0-21:3..."
229,AZ,"Food Trucks, Food, Restaurants, Mexican",4.5,VJVEDDJKccmrW1fhJCWmsQ,104,Taqueria La Hacienda,254 E Buckeye Rd,Phoenix,"{'Monday': '8:30-0:0', 'Tuesday': '8:30-0:0', ..."
230,AZ,"Restaurants, American (New), Breakfast & Brunch",4.0,b1m_PK-ggGR3CmBjdIyqkQ,139,Wrights at The Arizona Biltmore,2400 E Missouri Ave,Phoenix,"{'Monday': '17:30-21:30', 'Tuesday': '17:30-21..."
